In [1]:
import energy_consumption as ec
import json
import math
import numpy as np
import json
from pprint import pprint

In [2]:
# Define vehicle characteristics
with open("vehicle_data.json", "r") as file:
    vehicle_data = json.load(file)
# Define static data
static_data = {
    "air_dens": 1.225,  # kg/m³
    "grav_acc": 9.81,  # m/s²
}

# Define route segments
segments = [
    {
        "segment_name": "Flat start",
        "points": ["point1", "point2"],
        "coords": [-2.6080503, 51.4562142, -2.6080419, 51.4562768],
        "climb": 0,
        "distance": 50,
        "avg_incline_angle": 0,
        "stop_start": True,  # Starting from a stop
    },
    {
        "segment_name": "Gentle uphill",
        "points": ["point2", "point3"],
        "coords": [-2.6080419, 51.4562768, -2.6080335, 51.4563394],
        "climb": 1.5,
        "distance": 70,
        "avg_incline_angle": 2.5,
        "stop_start": False,  # Continuing from previous segment
    },
    {
        "segment_name": "Steep uphill",
        "points": ["point3", "point4"],
        "coords": [-2.6080335, 51.4563394, -2.6080251, 51.4564020],
        "climb": 4,
        "distance": 60,
        "avg_incline_angle": 6.8,
        "stop_start": False,  # Continuing from previous segment
    },
    {
        "segment_name": "Traffic light stop",
        "points": ["point4", "point5"],
        "coords": [-2.6080251, 51.4564020, -2.6080167, 51.4564646],
        "climb": 0,
        "distance": 80,
        "avg_incline_angle": 0,
        "stop_start": True,  # Stopped at traffic light
    },
    {
        "segment_name": "Downhill",
        "points": ["point5", "point6"],
        "coords": [-2.6080167, 51.4564646, -2.6080083, 51.4565272],
        "climb": -3,
        "distance": 100,
        "avg_incline_angle": -3.0,
        "stop_start": False,  # Continuing from previous segment
    }
]

# E-bike specifications
target_velocity = 5.5  # m/s (approx. 20 km/h)
max_motor_power = vehicle_data['max_motor_power']  # Watts (EU legal limit)
motor_efficiency = 0.85  # 85% efficiency

# Process the route
results = ec.process_route_segments(
    segments, vehicle_data, static_data, 
    target_velocity, max_motor_power, motor_efficiency
)

# Display results
print("\nE-BIKE ENERGY CONSUMPTION SIMULATION RESULTS\n" + "="*45)
print(f"Target velocity: {target_velocity} m/s ({target_velocity*3.6:.1f} km/h)")
print(f"Maximum motor power: {max_motor_power} Watts")
print(f"Motor efficiency: {motor_efficiency*100:.1f}%")
print(f"Vehicle mass: {vehicle_data['mass']} kg\n")

total_energy = 0
total_distance = 0
total_time = 0

for i, (segment, result) in enumerate(zip(segments, results)):
    print(f"SEGMENT {i+1}: {segment['segment_name']}")
    print(f"  Distance: {segment['distance']:.1f} m")
    print(f"  Incline: {segment['avg_incline_angle']:.1f}°")
    print(f"  Initial velocity: {result['initial_velocity']:.2f} m/s ({result['initial_velocity']*3.6:.1f} km/h)")
    print(f"  Final velocity: {result['final_velocity']:.2f} m/s ({result['final_velocity']*3.6:.1f} km/h)")
    print(f"  Acceleration: {result['acceleration']:.3f} m/s²")
    
    # Get energy consumption
    if "total_energy" in result:
        energy = result["total_energy"]
        time = result["total_time"]
        print(f"  Time: {time:.2f} seconds")
        print(f"  Energy: {energy:.2f} Wh")
        
        # Show breakdown for acceleration and constant phases
        accel_phase = result["acceleration_phase"]
        const_phase = result["constant_phase"]
        print(f"    • Acceleration phase: {accel_phase['distance']:.1f}m, {accel_phase['time']:.2f}s, {accel_phase['energy']:.2f} Wh")
        print(f"    • Constant phase: {const_phase['distance']:.1f}m, {const_phase['time']:.2f}s, {const_phase['energy']:.2f} Wh")
    else:
        energy = result["energy_consumption"]
        time = result["time"]
        print(f"  Time: {time:.2f} seconds")
        print(f"  Energy: {energy:.2f} Wh")
    
    print(f"  Target speed reached: {'Yes' if result['reached_target'] else 'No'}")
    print("")
    
    total_energy += energy
    total_distance += segment['distance']
    total_time += time if isinstance(time, (int, float)) else 0

print("="*45)
print(f"TOTAL ROUTE SUMMARY:")
print(f"  Total distance: {total_distance:.1f} m")
print(f"  Total time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
print(f"  Total energy consumption: {total_energy:.2f} Wh")
print(f"  Energy consumption per km: {total_energy/(total_distance/1000):.2f} Wh/km")
print("="*45)


E-BIKE ENERGY CONSUMPTION SIMULATION RESULTS
Target velocity: 5.5 m/s (19.8 km/h)
Maximum motor power: 3000 Watts
Motor efficiency: 85.0%
Vehicle mass: 200 kg

SEGMENT 1: Flat start
  Distance: 50.0 m
  Incline: 0.0°
  Initial velocity: 0.00 m/s (0.0 km/h)
  Final velocity: 5.50 m/s (19.8 km/h)
  Acceleration: 0.302 m/s²
  Time: 18.18 seconds
  Energy: 1.37 Wh
  Target speed reached: Yes

SEGMENT 2: Gentle uphill
  Distance: 70.0 m
  Incline: 2.5°
  Initial velocity: 5.50 m/s (19.8 km/h)
  Final velocity: 5.50 m/s (19.8 km/h)
  Acceleration: 0.000 m/s²
  Time: 12.73 seconds
  Energy: 3.16 Wh
  Target speed reached: No

SEGMENT 3: Steep uphill
  Distance: 60.0 m
  Incline: 6.8°
  Initial velocity: 5.50 m/s (19.8 km/h)
  Final velocity: 5.50 m/s (19.8 km/h)
  Acceleration: 0.000 m/s²
  Time: 10.91 seconds
  Energy: 5.59 Wh
  Target speed reached: No

SEGMENT 4: Traffic light stop
  Distance: 80.0 m
  Incline: 0.0°
  Initial velocity: 0.00 m/s (0.0 km/h)
  Final velocity: 5.50 m/s (19.8 